Na temelju odabranih podataka iz popisa, te izbornih podataka, može se izračunati sličnost između gradova/općina

In [1]:
import pandas as pd
import geopandas as gpd

Učitavanje geopodataka

In [2]:
gdf = gpd.read_file("data/administrative_units/gradovi_opcine_zupanije.geojson")
gdf['Županija'] = gdf['text_right'].str.strip()
gdf['Grad/općina'] = gdf['text_left'].str.strip()

In [3]:
def create_static_map(df, color_column_name):
    plot = df.plot(
        column=color_column_name, 
        legend=True, 
        figsize=(15, 15)
    )
    plot.set_title(color_column_name, fontsize=14)

    # Filter out rows with missing geometries
    df = df.dropna(subset=['geometry'])

Učitavanje onih podataka iz popisa koje želimo iskoristiti u analizi

In [ ]:
census_percentages = []

bračni_status = pd.read_csv("data/census/bračni_status.csv")
bračni_status = bračni_status[bračni_status['Starost'] == 'Ukupno']
bračni_status_percentages = bračni_status.columns[16:26]
bračni_status_percentages =list(bračni_status_percentages)
census_percentages += bračni_status_percentages

br_djece = pd.read_csv("data/census/broj_živorođene_djece.csv")
br_djece = br_djece[br_djece['Starost'] == 'Ukupno']
br_djece_percentages = br_djece.columns[[17] + list(range(19, 30))]
br_djece_percentages =list(br_djece_percentages)
census_percentages += br_djece_percentages

dob = pd.read_csv("data/census/dob.csv")
dob = dob[dob['Spol'] == 'sv.']
dob_percentages = dob.columns[[14] + [18] +[23] + [25]]
dob_percentages =list(dob_percentages)
census_percentages += dob_percentages

ekonomska_aktivnost = pd.read_csv("data/census/ekonomska_aktivnost.csv")
ekonomska_aktivnost = ekonomska_aktivnost[(ekonomska_aktivnost['Spol'] == 'sv.') & (ekonomska_aktivnost['Starost'] == 'Ukupno')]
ekonomska_aktivnost_percentages = ekonomska_aktivnost.columns[[16] + [17] + list(range(19,25))]
ekonomska_aktivnost_percentages =list(ekonomska_aktivnost_percentages)
census_percentages += ekonomska_aktivnost_percentages

mjesto_rođenja = pd.read_csv("data/census/mjesto_rođenja_stanovanja.csv")
mjesto_rođenja = mjesto_rođenja[mjesto_rođenja['Spol'] == 'sv.']
mjesto_rođenja['Rođeni u gradu/općini stanovanja %'] = (mjesto_rođenja['U mjestu stanovanja'] + mjesto_rođenja['U drugom naselju istoga grada/općine2)']) / mjesto_rođenja['Ukupan broj stanovnika'] * 100
mjesto_rođenja_percentages = [col for col in mjesto_rođenja.columns[list(range(22,25)) + list(range(26, 33))] if col.endswith('%')]
mjesto_rođenja_percentages =list(mjesto_rođenja_percentages)
mjesto_rođenja_percentages.append('Rođeni u gradu/općini stanovanja %')
census_percentages += mjesto_rođenja_percentages

narodnost = pd.read_csv("data/census/narodnost.csv")
narodnost_percentages = [col for col in narodnost.columns[5:62] if col.endswith('%')]
narodnost_percentages =list(narodnost_percentages)
census_percentages += narodnost_percentages

područje_djelatnosti = pd.read_csv("data/census/područje_djelatnosti.csv")
područje_djelatnosti = područje_djelatnosti[(područje_djelatnosti['Starost'] == 'Ukupno') & (područje_djelatnosti['Spol'] == 'sv.')]
područje_djelatnosti_percentages = [col for col in područje_djelatnosti.columns if col.endswith('%')]
područje_djelatnosti_percentages =list(područje_djelatnosti_percentages)
census_percentages += područje_djelatnosti_percentages

položaj_u_zaposlenju = pd.read_csv("data/census/položaj_u_zaposlenju.csv")
položaj_u_zaposlenju = položaj_u_zaposlenju[(položaj_u_zaposlenju['Spol'] == 'sv.') & (položaj_u_zaposlenju['Starost'] == 'Ukupno')]
položaj_u_zaposlenju_percentages = položaj_u_zaposlenju.columns[[12] + list(range(14,19))]
položaj_u_zaposlenju_percentages =list(položaj_u_zaposlenju_percentages)
census_percentages += položaj_u_zaposlenju_percentages

vjera = pd.read_csv("data/census/vjera.csv")
vjera_percentages = [col for col in vjera.columns[5:28] if col.endswith('%')]
vjera_percentages =list(vjera_percentages)
census_percentages += vjera_percentages

završena_škola = pd.read_csv("data/census/završena_škola.csv")
završena_škola = završena_škola[(završena_škola['Spol'] == 'sv.') & (završena_škola['Starost'] == 'Ukupno')]
završena_škola_percentages = završena_škola.columns[list(range(15,20)) + list(range(21,25))]
završena_škola_percentages =list(završena_škola_percentages)
census_percentages += završena_škola_percentages

census_df = (
    bračni_status
    .merge(br_djece, on=['Županija', 'Grad/općina'], how='inner', suffixes=('bračni_status', '_br_djece'))
    .merge(dob, on=['Županija', 'Grad/općina'], how='inner', suffixes=('', '_dob'))
    .merge(ekonomska_aktivnost, on=['Županija', 'Grad/općina'], how='inner', suffixes=('', '_ekonomska_aktivnost'))
    .merge(mjesto_rođenja, on=['Županija', 'Grad/općina'], how='inner', suffixes=('', '_mjesto_rođenja'))
    .merge(narodnost, on=['Županija', 'Grad/općina'], how='inner', suffixes=('', '_narodnost'))
    .merge(područje_djelatnosti, on=['Županija', 'Grad/općina'], how='inner', suffixes=('', '_područje_djelatnosti'))
    .merge(položaj_u_zaposlenju, on=['Županija', 'Grad/općina'], how='inner', suffixes=('', '_položaj_u_zaposlenju'))
    .merge(vjera, on=['Županija', 'Grad/općina'], how='inner', suffixes=('', '_vjera'))
    .merge(završena_škola, on=['Županija', 'Grad/općina'], how='inner', suffixes=('', '_završena_škola'))
)
df = gdf.merge(census_df, on=['Županija', 'Grad/općina'], how='inner')

print(census_percentages)

Učitavanje izbornih rezultata

In [ ]:
election_results = pd.read_csv("data/election_results/kombinirani_rezultati.csv")

party_percentages = election_results.columns[34:61] 
party_percentages = list(party_percentages)
print(party_percentages)

election_results['Županija'] = election_results['Županija'].str.strip()  # Strip any extra spaces
election_results['Grad/općina'] = election_results['Grad/općina/država'].str.strip()

df = gdf.merge(election_results, on=['Županija', 'Grad/općina'], how='inner').merge(census_df, on=['Županija', 'Grad/općina'], how='inner')
features = party_percentages + census_percentages
print(features)

Izračun indeksa sličnosti

In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

data = df[features]
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)
similarity_matrix = np.corrcoef(scaled_data)

selected_row_index = df[df['Grad/općina'] == 'SPLIT'].index[0]  # Get the index of the matching row
similarities = similarity_matrix[selected_row_index]

# Add the similarities to the DataFrame for easy interpretation
df['Similarity'] = similarities

create_static_map(df, 'Similarity')